In [1]:
from scrap import Scraper

In [2]:
url = "https://admissions.comsats.edu.pk/Home/AboutComsats"
scrp = Scraper(url)

In [3]:
print(scrp.scrape_website())

COMSATS University Islamabad (CUI), a leading Degree Awarding Institution of higher education in Pakistan, is among the Centers of Excellence of Commission on Science and Technology for Sustainable Development in the South (COMSATS) www.comsats.org. Established in 1998, CUI is one of the fast growing research - based institutions in Pakistan, with a wide range of academic programs (ranging from basic sciences to cutting edge emerging technologies) and a network of inter-disciplinary research centers making it an ideal place for higher studies leading to MS and PhD degrees. Since its establishment, CUI has made multi-faceted growth in terms of campuses (currently it has eight fully functional campuses), number of students, faculty members, academic programs, research output, and public outreach, to accomplish its three-fold stated mission, i.e., Research and Discovery, Teaching and Learning, and Outreach and Public Service, which makes it a popular choice for undergraduate as well as gr

In [4]:
from RAG_QnA import RAG_Model

In [5]:
rag_model = RAG_Model()

2024-07-29 12:02:06.676025: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 12:02:06.698098: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 12:02:06.704540: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-29 12:02:07.968337: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/junaid-ul-hassan/.cache/huggingface/token
Login successful


In [ ]:
print(rag_model.generateResponse(prompt="What is Admission dates for Comsates?"))

Number of requested results 50 is greater than number of elements in index 6, updating n_results = 6
